In [28]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
import random

In [2]:
class Digit:
    def __init__(self):
        self.choices = [x for x in range(10)]
    def guess(self):
        return random.choice(self.choices)
    def reinforce(self,value):
        self.choices.append(int(value))
    def remove(self,value):
        while int(value) in self.choices:
            self.choices.pop(self.choices.index(int(value)))
    def discourage(self,value):
        if int(value) in self.choices:
            self.choices.pop(self.choices.index(int(value)))

In [3]:
def enter_guess(guess):
    browser.find_by_id('guess-text').fill(guess)
    browser.find_by_id('guess-submit').click()
    
def retrieve_feedback():
    results = pd.read_html(browser.html,index_col=0)[0]
    return results

def process_answer(guess,debug=False):
    digits = [int(character) for character in guess]
    results = retrieve_feedback()
    if debug:
        print(results)
    most_recent = results.index.max()
    rdrp = results['Digit & Place'].loc[most_recent]
    rdwp = results['Digit, not Place'].loc[most_recent]
    
    if rdrp == 0 and rdwp == 0:
#         remove all digits of guess from each digits 'maybe' list and add to 'no' list
        for guess_digit in guess:
            for answer_digit in answer:
                answer_digit.remove(guess_digit)
    elif rdrp == 0:
        for index,guess_digit in enumerate(guess):
            answer[index].remove(guess_digit)
            
    if rdwp == 0:
        for g_index,guess_digit in enumerate(guess):
            for a_index,answer_digit in enumerate(answer):
                if g_index != a_index:
                    answer_digit.discourage(guess_digit)
            
    if rdwp > 0:
        for guess_digit in guess:
            for answer_digit in answer:
                answer_digit.reinforce(guess_digit)
                
    if rdrp > 0:
        for index,guess_digit in enumerate(guess):
            for x in range(rdrp):
                answer[index].reinforce(guess_digit)
            

In [30]:
# Parameters 
bot_name = 'beepBoopBoop'
patience = 20
url = 'https://ucbe-mastermind.herokuapp.com/'
answer = [Digit() for x in range(4)]

# Visit page and enter bot_name
browser = Browser("chrome",headless=True)
browser.visit(url)
browser.find_by_id('player-name').fill(bot_name)

guessing = True
guess_count = 0
past_guesses = []
while guessing:
    this_guess = [str(digit.guess()) for digit in answer]
    while int(''.join(this_guess)) in past_guesses:
        this_guess = [str(digit.guess()) for digit in answer]
    enter_guess(this_guess)
    process_answer(this_guess)
    feedback = retrieve_feedback()
    past_guesses.append(int(''.join(this_guess)))
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        guessing = False
    guess_count += 1 
    if guess_count >= patience:
        guessing = False
        browser.find_by_id('forfeit').click()
        
print(feedback)
browser.quit()

    Guess  Digit & Place  Digit, not Place
20    492              1                 2
19    400              2                 0
18    190              1                 1
17   9200              1                 1
16   1491              2                 0
15   9400              2                 0
14    440              2                 0
13   9910              1                 1
12   9410              2                 1
11   9490              2                 0
10   4440              2                 0
9    4244              0                 2
8    4090              1                 1
7    8588              0                 0
6     209              0                 2
5    4044              0                 2
4      85              0                 1
3       9              0                 1
2    7366              0                 0
1    9038              0                 1


In [13]:
# restart game
browser.click_link_by_partial_text('Play')

In [81]:
import itertools
import numpy as np

# Parameters 
bot_name = 'bruteForce'
patience = 20
url = 'https://ucbe-mastermind.herokuapp.com/'
answer = [Digit() for x in range(4)]

# Visit page and enter bot_name
browser = Browser("chrome",headless=True)
browser.visit(url)
browser.find_by_id('player-name').fill(bot_name)

guessing = True
guess_count = 0
past_guesses = []
# Phase 1: identify digits
phase_one = np.random.choice([x for x in range(10)],10,replace=False)
for digit in phase_one:
    this_guess = [str(digit) for x in range(4)]
    enter_guess(this_guess)
    feedback = retrieve_feedback()
    past_guesses.append(int(''.join(this_guess)))
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        break
    guess_count += 1 
    if guess_count >= patience:
        browser.find_by_id('forfeit').click()
        break
    if feedback[['Digit & Place', 'Digit, not Place']].sum().sum() == 4:
        break
# Phase 2: determine order
unordered = []
hits = feedback.loc[feedback[['Digit & Place', 'Digit, not Place']].sum(axis=1) > 0]
for index in hits.index:
    for x in range(hits['Digit & Place'].loc[index]):
        unordered.append(str(int(hits['Guess'].loc[index]/1111)))
permutations = []
for permutation in itertools.permutations(unordered, 4):
    permutations.append(''.join([str(x) for x in permutation]))
list(pd.Series(permutations).unique())
phase_two = np.random.choice(list(pd.Series(permutations).unique()),len(list(pd.Series(permutations).unique())),replace=False)
for this_guess in phase_two:
    enter_guess(this_guess)
    feedback = retrieve_feedback()
    past_guesses.append(int(''.join(this_guess)))
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        break
    guess_count += 1 
    if guess_count >= patience:
        browser.find_by_id('forfeit').click()
        break
print(feedback)
browser.quit()

    Guess  Digit & Place  Digit, not Place
13   8622              4                 0
12   2862              1                 3
11   8262              2                 2
10   8888              1                 0
9    9999              0                 0
8    2222              2                 0
7       0              0                 0
6    6666              1                 0
5    5555              0                 0
4    3333              0                 0
3    4444              0                 0
2    7777              0                 0
1    1111              0                 0


In [79]:
unordered

[9.0, 2.0, 6.0, 1.0]

In [177]:
import time 
# Parameters 
bot_name = 'eliminate'
patience = 20
url = 'https://ucbe-mastermind.herokuapp.com/'
answer = [Digit() for x in range(4)]

# Visit page and enter bot_name
browser = Browser("chrome",headless=False)
browser.visit(url)
browser.find_by_id('player-name').fill(bot_name)

guess_space = 10000
possibilities = pd.DataFrame({
    'Guess':[x for x in range(guess_space)],
    1:[int(np.floor(x/1000)) for x in range(guess_space)],
    2:[int(np.floor(x/100)%10) for x in range(guess_space)],
    3:[int(np.floor(x/10)%10) for x in range(guess_space)],
    4:[x%10 for x in range(guess_space)],
    'all':[[int(np.floor(x/1000)),
            int(np.floor(x/100)%10),
            int(np.floor(x/10)%10),
            x%10] for x in range(guess_space)]
    })

guessing = True
guess_count = 0
past_guesses = []
progression = []
while guessing:
    this_key = random.choice(possibilities.index)
    this_guess = [str(x) for x in possibilities.loc[this_key,'all']]
    enter_guess(this_guess)
    process_answer(this_guess)
    feedback = retrieve_feedback()
    past_guesses.append(int(''.join(this_guess)))
    
    n_perfect = feedback['Digit & Place'].as_matrix()[0]
    n_almost = feedback['Digit, not Place'].as_matrix()[0]
    
    count_possibilities = 0
    for possibility in possibilities.index:
        if possibility != this_key:
            candidate = list(possibilities.loc[possibility,'all'])
            was_guessed = list(possibilities.loc[this_key,'all'])
            perfect_tally = 0
            almost_tally = 0
            
            b = candidate
            a = was_guessed
            
            for index,x in enumerate(a):
                if x == b[index]:
                    perfect_tally += 1
                    a[index] = -1
                    b[index] = -1
            for x in a:
                if (x != -1) & (x in b):
                    almost_tally += 1
                    b[b.index(x)] = -1  
            if (perfect_tally == n_perfect) & (almost_tally == n_almost):
                count_possibilities += 1
            else:
                possibilities.drop([possibility],inplace=True)
    print(f'{count_possibilities} remaining')
    progression.append(list(possibilities.index))
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        guessing = False
    if guess_count >= patience:
        guessing = False
        browser.find_by_id('forfeit').click()
    possibilities.drop([this_key],inplace=True)
    guess_count += 1
        
print(feedback)
# time.sleep(30)
# browser.quit()

1296 remaining
132 remaining
18 remaining
8 remaining
3 remaining
2 remaining
1 remaining
0 remaining
   Guess  Digit & Place  Digit, not Place
8   7408              4                 0
7   7808              3                 0
6   7508              3                 0
5   7008              3                 0
4   2778              1                 1
3   7780              1                 2
2   8702              1                 2
1   9163              0                 0


In [178]:
browser.quit()